In [1]:
using JuMP, Gurobi, DataFrames, CSV, StatsBase, MLDataUtils, Random

### Sets:

(1) $ i=1, \cdots, I$: Patient

(2) $ j=1, \cdots, J$: Room

(3) $ t=1, \cdots, T$: Day

### Parameters:


(1) $l_{i}$: Expected length of stay (LOS) of ${patient}_{i}$

(2) $g_{i}$: Gender of ${patient}_{i}$ such that: $g_{i} = 1$ if ${patient}_{i}$ is female, and $-1$ otherwise.

(3) $c_{j}$: Capacity (beds) of ${room}_{j}$


### Decision Variables:

(1) $x_{i j t}$: Binary variable such that $x_{i j t} = 1$ if ${patient}_{i}$ stays in ${room}_{j}$ on ${day}_{t}$ and $0$ otherwise.

(2) $y_{i j}$: Binary variable such that $y_{i j} = 1$ if ${patient}_{i}$ is assigned to ${room}_{j}$ and $0$ otherwise.


### Formulation:

$
\begin{array}{ll}
{\text { maximize }} & {\sum_{i} \sum_{j} \sum_{t} x_{i j t}} \quad {\text {total number of beds utilized}} \\ {\text { subject to }} & {\sum_{j} y_{i j} \leq 1 \quad \forall i} \quad {\text {a patient gets assigned for at most 1 room}} \\ {} & x_{i j t} \leq y_{i j} \quad \forall i,j,t \quad {\text {patients only stay in the room assigned to them}} \\ {} & {\sum_{j} \sum_{t} x_{i j t}} = l_{i} \cdot {\sum_{j} y_{i j}} \quad \forall i \quad {\text {if assigned, enforce LOS; if not, enforce 0}} \\ {} & ({\sum_{j} \sum_{t=2}^{T} |x_{i j t} - x_{i j t-1}|}) + ({\sum_{j} x_{i j 1} + x_{i j T}}) \leq 2 \quad \forall i \quad {\text {stay needs to be consecutive}} \\ {} & {\sum_{i} x_{i j t}} \leq c_{j} \quad \forall j,t \quad {\text {room capacity constraint}} \\ {} & {\sum_{i} x_{i j t}} \leq |{\sum_{i} g_{i} \cdot x_{i j t}}| \quad \forall j,t \quad {\text {all patients in same room have same gender for a given day}}
\end{array}
$

### Linear Formulation:

$
\begin{array}{ll}
{\text { maximize }} & {\sum_{i} \sum_{j} \sum_{t} x_{i j t}} \\ {\text { subject to }} & {\sum_{j} y_{i j} \leq 1 \quad \forall i} \\ {} & x_{i j t} \leq y_{i j} \quad \forall i,j,t \\ {} & {\sum_{j} \sum_{t} x_{i j t}} = l_{i} \cdot {\sum_{j} y_{i j}} \quad \forall i \\ {} & {\sum_{j} \sum_{t=2}^{T} s_{i j t}} + {\sum_{j} x_{i j 1} + x_{i j T}} \leq 2 \quad \forall i \\ {} & s_{i j t} \geq x_{i j t} - x_{i j t-1} \quad \forall i,j,t \\ {} & s_{i j t} \geq x_{i j t-1} - x_{i j t}  \quad \forall i,j,t \\ {} & {\sum_{i} x_{i j t}} \leq c_{j} \quad \forall j,t \\ {} & {\sum_{i} x_{i j t}} \leq {\sum_{i} g_{i} \cdot x_{i j t}} + p_{j t} \cdot M \quad \forall j,t \\ {} & {\sum_{i} x_{i j t}} \leq -{\sum_{i} g_{i} \cdot x_{i j t}} + (1-p_{j t}) \cdot M \quad \forall j,t \\ {} & p_{j t} \in \{0, 1\} \quad \forall j,t
\end{array}
$

In [2]:
# Mini Data
LOS = [1,2,3,5,2,7,9,3,4,1] # patient length of stay
CAP = [1,3] # room capacity
G = [1, 1, 1, 1, 1, -1, -1, -1, -1, -1]; # gender, 1 = female, -1 = male

In [3]:
# Without Gender Constraint
I = length(LOS)
J = length(CAP)

model = Model(solver=GurobiSolver(OutputFlag=0))

@variable(model, x[1:I, 1:J, 1:T], Bin)
@variable(model, y[1:I, 1:J], Bin)
@variable(model, s[1:I, 1:J, 1:T])

@constraint(model, [i=1:I], sum(y[i,j] for j=1:J) <= 1)
@constraint(model, [i=1:I, j=1:J, t=1:T], x[i,j,t] <= y[i,j])
@constraint(model, [i=1:I], sum(x[i,j,t] for j=1:J, t=1:T) == LOS[i]*sum(y[i,j] for j=1:J))

@constraint(model, [i=1:I], sum(s[i,j,t] for j=1:J, t=2:T) + sum(x[i,j,1]+x[i,j,T] for j=1:J) <= 2)
@constraint(model, [i=1:I, j=1:J, t=2:T], s[i,j,t] >= x[i,j,t] - x[i,j,t-1])
@constraint(model, [i=1:I, j=1:J, t=2:T], s[i,j,t] >= x[i,j,t-1] - x[i,j,t])

@constraint(model, [j=1:J, t=1:T], sum(x[i,j,t] for i=1:I) <= CAP[j])
@objective(model, Max, sum(x[i,j,t] for i=1:I, j=1:J, t=1:T))
solve(model)
x = getvalue(x)
y = getvalue(y)
getobjectivevalue(model)

Academic license - for non-commercial use only


37.0

In [4]:
# With Gender Constraint
I = length(LOS)
J = length(CAP)

model = Model(solver=GurobiSolver(OutputFlag=0))

@variable(model, x[1:I, 1:J, 1:T], Bin)
@variable(model, y[1:I, 1:J], Bin)
@variable(model, s[1:I, 1:J, 1:T])
@variable(model, p[1:J, 1:T], Bin)

@constraint(model, [i=1:I], sum(y[i,j] for j=1:J) <= 1)
@constraint(model, [i=1:I, j=1:J, t=1:T], x[i,j,t] <= y[i,j])
@constraint(model, [i=1:I], sum(x[i,j,t] for j=1:J, t=1:T) == LOS[i]*sum(y[i,j] for j=1:J))

@constraint(model, [i=1:I], sum(s[i,j,t] for j=1:J, t=2:T) + sum(x[i,j,1]+x[i,j,T] for j=1:J) <= 2)
@constraint(model, [i=1:I, j=1:J, t=2:T], s[i,j,t] >= x[i,j,t] - x[i,j,t-1])
@constraint(model, [i=1:I, j=1:J, t=2:T], s[i,j,t] >= x[i,j,t-1] - x[i,j,t])

@constraint(model, [j=1:J, t=1:T], sum(G[i]*x[i,j,t] for i=1:I) + p[j,t]*10000 >= sum(x[i,j,t] for i=1:I))
@constraint(model, [j=1:J, t=1:T], -sum(G[i]*x[i,j,t] for i=1:I) + (1-p[j,t])*10000 >= sum(x[i,j,t] for i=1:I))

@constraint(model, [j=1:J, t=1:T], sum(x[i,j,t] for i=1:I) <= CAP[j])
@objective(model, Max, sum(x[i,j,t] for i=1:I, j=1:J, t=1:T))
solve(model)
x = getvalue(x)
y = getvalue(y)
getobjectivevalue(model)

Academic license - for non-commercial use only


35.0